IV. Пакети python та плагіни napari
===================================

_Image Analysis with Python and Napari, Bioinformatics for Ukraine course, 6-24 October 2025, Kyiv, Ukraine._

_© Borys Olifirov, 2025_

__План:__
- Структура пакетів Python
- Оформлення набору функцій як пакету
- Встановлення власного пакету
- Перетворення пакету в плагін napari

---

In [ ]:
import numpy as np
import skimage  # scikit-image
import matplotlib.pyplot as plt
import napari